## Radarbanten.co.id

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime
import threading
import re

In [2]:
jumlah_index = 1
threads_link = []
links = []
results = []
threads = []

In [3]:
def scrape_links(page_number):
    url=f"https://www.radarbanten.co.id/category/kota-serang/page/{page_number}/"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    articles = soup.find_all('h3',{"class": "jeg_post_title"})
    
    page_links = []
    for article in articles:
        link = article.find('a')['href']
        page_links.append(link)
    
    print(f"Scraped {len(page_links)} links from page {page_number}")
    return page_links

In [4]:
for page_number in range(1, jumlah_index + 1):
    thread = threading.Thread(target=lambda p=page_number: links.extend(scrape_links(p)))
    thread.start()
    threads_link.append(thread)

for thread in threads_link:
    thread.join()
print("Total Links:", len(links))

Scraped 18 links from page 1
Total Links: 18


In [5]:
def scrape_url(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')        
            # Judul Berita
            title_elem = soup.find('h1')
            if title_elem:
                title_text = title_elem.text
            else:
                title_text = "Title not found"
            # Author berita
            author_elem = soup.find('div', {"class": "jeg_meta_author"})
            if author_elem:
                author_text = author_elem.find('a')
                author_text = author_text.text
            else:
                author_text = "Author not found"
                     
            # tanggal berita
            date_elem = soup.find('div', {"class": "jeg_meta_date"})
            if date_elem:
                date_text = date_elem.find('a')
                date_text = date_text.text
            else:
                date_text = "Date not found"
            # Category berita
            category_elements = soup.find('span', {"class": "breadcrumb_last_link"})
            if category_elements:
                category_text = category_elements.find('a')
                category_text= category_text.text
            else:
                category_text = "Category not found"
            #     # Content Berita
            body_elem = soup.find('div', {"class": "entry-content with-share"})
            
            if body_elem:
                content_elem = body_elem.find_all('p')
                content_text = ""
                for p in content_elem:
                    content_text += p.text.strip() + "\n"
                
                if content_text.strip():
                    content_text=content_text
                else:
                    content_text ="Content not found"
            else:
                content_text ="Content not found"
            nama_berita_match = re.search(r'https://www\.(\w+)\.co\.id/', url)
            if nama_berita_match:
                nama_berita = nama_berita_match.group(1)
            else:
                nama_berita = "Nama_berita not found"


            results.append({'title': title_text,
                            'author' : author_text,
                            'category':category_text,
                            'date': date_text,
                            'content' : content_text,
                            'nama_berita' : nama_berita,
                            'region':'banten',
                            'link' : url})
        else:
            print(f"Failed to retrieve data from {url}")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL '{url}': {e}")
    except Exception as e:
        print(f"Error processing URL '{url}': {e}")

In [6]:
for url in links:
    thread = threading.Thread(target=scrape_url, args=(url,))
    thread.start()
    threads.append(thread)
    
for thread in threads:
    thread.join()

In [7]:
df = pd.DataFrame(results)
print("Total Hasil ",len(results))
df.head(10)

Total Hasil  18


,title,author,category,date,content,nama_berita,region,link
0,"Luar Biasa, Sekolah di Banten Ini Mewakili Ind...",Rostinah,Utama,29-09-2023 09:27:59,"SERANG, RADARBANTEN.CO.ID – Kabar membanggakan...",radarbanten,banten,https://www.radarbanten.co.id/luar-biasa-sekol...
1,Ada Festival Balon Udara ala Cappadocia di Kot...,Rostinah,Kombis,26-09-2023 18:09:01,"SERANG, RADARBANTEN.CO.ID – Mau lihat festival...",radarbanten,banten,https://www.radarbanten.co.id/ada-festival-bal...
2,"Larang PKL Berjualan di Stadion Maulana Yusuf,...",Nahrul Muhilmi,Bisnis,26-09-2023 15:13:48,"SERANG, RADARBANTEN.CO.ID – Walikota Serang, S...",radarbanten,banten,https://www.radarbanten.co.id/larang-pkl-berju...
3,"Pembangunan Awning Dianggap Ilegal, Pengelola ...",Nahrul Muhilmi,Bisnis,26-09-2023 16:06:49,"SERANG, RADARBANTEN.CO.ID – Calon pengelola Wi...",radarbanten,banten,https://www.radarbanten.co.id/pembangunan-awni...
4,Kasus Dugaan Korupsi Dana Hibah 2022: Hampir L...,Fahmi,Hukum,25-09-2023 21:19:45,"SERANG, RADARBANTEN.CO.ID – Empat pengurus KON...",radarbanten,banten,https://www.radarbanten.co.id/kasus-dugaan-kor...
5,"Banten Darurat Kekeringan, 419 Desa Alami Kris...",Yusuf Permana,Berita Utama,26-09-2023 14:48:32,"SERANG,RADARBANTEN.CO.ID-Pemerintah Provinsi (...",radarbanten,banten,https://www.radarbanten.co.id/banten-darurat-k...
6,Pengelola Wisata Kuliner Stadion MY Sebut PKL ...,Nahrul Muhilmi,Kota Serang,26-09-2023 17:38:04,"SERANG,RADARBANTEN.CO.ID – Pengelola Wisata Ku...",radarbanten,banten,https://www.radarbanten.co.id/pengelola-wisata...
7,Kasus Proyek Jalan Pelabuhan Warnasari Cilegon...,Fahmi,Berita Utama,27-09-2023 19:17:40,"\nSERANG, RADARBANTEN.CO.ID – Sebanyak 63 saks...",radarbanten,banten,https://www.radarbanten.co.id/kasus-proyek-jal...
8,"Kepala Kemenag Banten Dilaporkan ke Kejati, Di...",Fahmi,Hukum,25-09-2023 17:17:57,"SERANG, RADARBANTEN.CO.ID – Kepala Kanwil Keme...",radarbanten,banten,https://www.radarbanten.co.id/kepala-kemenag-b...
9,Pengelola PKL Stadion Maulana Yusuf Bantah Ban...,Nahrul Muhilmi,Kota Serang,25-09-2023 21:35:48,"SERANG, RADARBANTEN.CO.ID – Pihak pengelola pe...",radarbanten,banten,https://www.radarbanten.co.id/pengelola-pkl-st...


# Save File 

In [8]:
# current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
# excel_file_name = f'../../tempat_hasil_daerah/radarBanten_{current_datetime}.xlsx'
# df.to_excel(excel_file_name, index=False)

# print(f'Data has been saved to {excel_file_name}')